In [18]:
from io import BytesIO
from PIL import Image
import datasets
import joblib
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModel
from sklearn.linear_model import LogisticRegression
import torch
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
from sklearn.metrics import classification_report
from deepface.extendedmodels import Age
from deepface.modules import modeling, detection, preprocessing
import tensorflow as tf

In [4]:
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def _load_age_model():
    # if not os.path.exists('/root/.deepface/weights'):
    #     os.mkdir('/root/.deepface')
    #     os.mkdir('/root/.deepface/weights')
    age_model = modeling.build_model("Age").model
    return age_model


In [19]:
def _predict_age(age_model, bgr_images):
    bgr_images = [preprocessing.resize_image(img=np.array(img_content), target_size=(224, 224)) for img_content in bgr_images]
    # Returns predictions of each batch - (batch_size, age_classes)
    # age_preds = age_model(np.array(bgr_images).squeeze())

    input_array = np.array(bgr_images).squeeze()
    with tf.device('/GPU:0'):  # TensorFlow syntax for forcing GPU
        age_preds = age_model(input_array, training=False)  # Important to set training=False for inference

    output_indexes = np.arange(101)

    # Multiply each age class probability by its corresponding age index and sum across the age axis
    apparent_ages = np.sum(age_preds * output_indexes, axis=1)

    return apparent_ages

In [15]:
age_model = _load_age_model()

In [16]:
def get_deepface_preds(sample):
    # import pdb; pdb.set_trace()

    # Age prediction
    # age_texts = [f"A person in the {c} age group" for c in ["0-2", "3-9", "10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "more than 70"]]
    # inputs = processor(text=age_texts, images=sample["image"], return_tensors="pt", padding=True).to(device)
    # outputs = model(**inputs)
    # logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    # age_pred = logits_per_image.argmax(dim=1) # we can take the argmax
    # sample["zs_age_clip"] = [int(gp) for gp in age_pred]
    age_pred = _predict_age(age_model, sample["image"])
    sample["deepface_pred"] = [int(gp) for gp in age_pred]
    return sample

In [21]:
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))

GPUs Available:  []


In [32]:
# Load training data
train_ds = datasets.load_dataset('HuggingFaceM4/FairFace', '1.25', split='train', verification_mode="no_checks")
# train_ds = train_ds.shuffle(seed=42).select([i for i in range(1_000)]) # Take only first 20_000 images
train_ds = train_ds.shuffle(seed=42)
train_ds = train_ds.map(get_deepface_preds, batched = True, batch_size=16)

Map:   0%|          | 0/86744 [00:00<?, ? examples/s]

In [ ]:
df_preds_train = np.array(train_ds["deepface_pred"])
np.save("deepace_age_preds_train_42.npy", df_preds_train)

: 

In [ ]:
# # Load validation data and test on this
# valid_ds = datasets.load_dataset('HuggingFaceM4/FairFace', '1.25', split="validation", verification_mode="no_checks")
# valid_ds = valid_ds.shuffle(seed=42)
# # valid_ds = valid_ds.shuffle(seed=42) 
# valid_ds = valid_ds.map(get_deepface_preds, batched = True, batch_size=16)

Map:   0%|          | 0/10954 [00:00<?, ? examples/s]

In [ ]:
# df_preds_val = np.array(valid_ds["deepface_pred"])

In [ ]:
# np.save("deepace_age_preds_val_42.npy", df_preds_val)

10954